In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import KNNImputer

In [2]:
pd.options.display.max_columns = None
present_day = pd.Timestamp('2023-01-01') # Fix the day considered "today" to keep training consistent

raw_train_data = pd.read_csv('train.csv', parse_dates=['host_since'])
y = raw_train_data['price']
raw_train_data

C:\Users\jyson\AppData\Local\Temp\ipykernel_14672\4034484329.py:4: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_train_data = pd.read_csv('train.csv', parse_dates=['host_since'])


,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,country_code,country,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,22267382,Modern and Cozy Large Studio in Brooklyn,Modern large studio with new amenities and app...,Our place is a little quiet sanctuary in the h...,Modern large studio with new amenities and app...,none,"BAM, Barclays, Brooklyn City Point, Fort Green...",NaN,"Subway: 2,3,4,5,A,C,B,Q,G",Washer/Dryer Dishwasher Internet Gym Roof Top ...,"Depending on the time of your visit, I'll be h...","- Please be respectful of our neighbors, no lo...",1910170,Katarina,2012-03-12,"Los Angeles, California, United States","Traveler, Artist, Philanthropist",NaN,NaN,NaN,f,Fort Greene,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Fort Greene,Brooklyn,Brooklyn,NY,11217.0,New York,US,United States,Loft,Entire home/apt,2,1.0,1.0,1.0,Real Bed,"{Wifi,""Air conditioning"",Kitchen,Gym,Breakfast...",NaN,145.0,2,$30.00,7,12,6,2018-01-01,2018-07-03,100.0,10.0,10.0,10.0,10.0,10.0,10.0,t,f,flexible,f,f,1,0.59
1,2473861,Royal Harlem TRIPLEX Home 5 Beds,Harlem is back and so gorgeous! Visit and expl...,Harlem is back and so gorgeous! Visit and expl...,Harlem is back and so gorgeous! Visit and expl...,none,HARLEM is a piece of real NY history overflowi...,HARLEM RESTAURANTS Red Rooster Harlem -- excel...,PUBLIC TRANSPORTATION: Conveniently near all p...,The WHOLE ENTIRE HOUSE,NaN,"Smoking, pets and unaccounted guests NOT permi...",8826175,Grover,2013-09-14,"New York, New York, United States",As the owners of the property we have realized...,within an hour,100%,NaN,f,Mid-Wilshire,4.0,"['email', 'phone', 'reviews', 'manual_offline']",t,t,Harlem,Manhattan,New York,NY,10027.0,New York,US,United States,House,Entire home/apt,8,1.0,3.0,5.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",NaN,175.0,3,$19.00,3,30,137,2014-04-15,2018-10-20,91.0,9.0,9.0,9.0,9.0,9.0,9.0,t,f,moderate,f,f,3,2.47
2,25079703,Sunny East Village Studio,"Clean, hip and well designed sun drenched East...",This is a rare East Village studio with it's h...,"Clean, hip and well designed sun drenched East...",none,East Village is one of the last remaining neig...,NaN,NaN,You'll have access to the entire space - it's ...,"Very responsive via phone call, text or email.",NaN,4383563,Zander,2012-12-11,"New York, New York, United States","New York City based, currently working as a te...",within a few hours,100%,NaN,f,East Village,1.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,East Village,Manhattan,New York,NY,10009.0,New York,US,United States,Apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,Heating,""S...",NaN,180.0,1,$0.00,2,1125,3,2018-07-26,2018-10-14,100.0,10.0,9.0,9.0,10.0,10.0,10.0,f,f,moderate,f,f,1,0.89
3,9342478,"Beautiful, airy, light-filled room","Private, spacious, comfortable room in 2-bed f...","Big closet, two big windows, tall ceiling and ...","Private, spacious, comfortable room in 2-bed f...",none,One block from Morgan L stop. Super cool area....,NaN,NaN,NaN,NaN,NaN,10406276,Kathleen,2013-12-03,"New York, New York, United States",Aus

In [3]:
'''
Clean up data
'''

'''
Could use later: host_neighbourhood, host_verifications, neighborhood_cleansed, neighborhood_group_cleansed,
state, square_feet, last_review
'''

# Returns a dataframe with relevant columns with data ready for preprocessing
def retrieve_features(raw_data):
    # TODO: Add all review_... columns
    # Grab relevant features and ID columns (i.e., non-text features)
#     feat_names = ['id', 'host_since', 'host_response_time', 
#                   'host_response_rate', 'host_listings_count', 'host_has_profile_pic', 
#                   'host_identity_verified', 'property_type', 'room_type', 'accommodates',
#                   'bathrooms', 'bedrooms', 'beds', 'bed_type', 'guests_included',
#                   'extra_people', 'minimum_nights', 'maximum_nights', 'number_of_reviews',
#                  'review_scores_rating', 'cancellation_policy']

    num_cols = ['id', 'host_since', 
                  'host_response_rate', 'host_listings_count', 'host_has_profile_pic', 
                  'host_identity_verified',
                  'bathrooms', 'bedrooms', 'beds',
                  'extra_people', 'minimum_nights', 'maximum_nights', 'number_of_reviews',
                 'review_scores_rating']
    
    cat_cols = ['host_response_time', 'property_type', 'room_type', 'accommodates', 'bed_type', 
                'guests_included', 'cancellation_policy']

    data = raw_data[num_cols + cat_cols].set_index('id')

    # Replace 't' with True and 'f' with False
    data = data\
            .replace(to_replace='t', value=True)\
            .replace(to_replace='f', value=False)\
    # Change percentages as strings in 'host_response_rate' to fraction from 0 to 1
    data['host_response_rate'] = data['host_response_rate'].str.extract(r"(^\d+)%").astype('float') / 100.

    # Change dollar values in 'extra_people' to float
    data['extra_people'] = data['extra_people'].str.extract(r"^\$(\d+)").astype('float')

    # Change 'host_since' into number of days hosting until present
    data['host_since'] = data['host_since'].apply(lambda x: (present_day - x).days)
    
    return data

In [22]:
train_data = retrieve_features(raw_train_data)

x_train, x_val, y_train, y_val = train_test_split(train_data, y)
train_data

,host_since,host_response_rate,host_listings_count,host_has_profile_pic,host_identity_verified,bathrooms,bedrooms,beds,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,host_response_time,property_type,room_type,accommodates,bed_type,guests_included,cancellation_policy
id,,,,,,,,,,,,,,,,,,,,
22267382,3947.0,NaN,1.0,True,True,1.0,1.0,1.0,30.0,7,12,6,100.0,NaN,Loft,Entire home/apt,2,Real Bed,2,flexible
2473861,3396.0,1.00,4.0,True,True,1.0,3.0,5.0,19.0,3,30,137,91.0,within an hour,House,Entire home/apt,8,Real Bed,3,moderate
25079703,3673.0,1.00,1.0,True,True,1.0,0.0,1.0,0.0,2,1125,3,100.0,within a few hours,Apartment,Entire home/apt,2,Real Bed,1,moderate
9342478,3316.0,NaN,1.0,True,True,1.0,1.0,1.0,0.0,3,1125,0,NaN,NaN,Apartment,Private room,1,Real Bed,1,flexible
4866426,3887.0,1.00,1.0,True,True,1.0,1.0,1.0,14.0,1,90,144,97.0,within a few hours,Apartment,Private room,2,Real Bed,1,flexible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3958032,3849.0,NaN,1.0,True,True,1.0,1.0,1.0,25.0,3,1125,19,87.0,NaN,Apartment,Private room,2,Real Bed,1,strict_14_with_grace_period
7987209,2690.0,NaN,3.0,True,True,3.0,4.0,4.0,0.0,3,365,86,95.0,NaN,Serviced apartment,Entire home/apt,8,Real Bed,1,strict_14_with_grace_period
796128,3895.0,NaN,1.0,True,True,NaN,1.0,1.0,0.0,180,365,0,NaN,NaN,House,Private room,1,Real Bed,1,flexible


In [17]:
# Preprocess data (mostly one-hot encoding). Could process later: bathrooms, bedrooms, beds
one_hot_cols = ['host_response_time', 'property_type', 'room_type', 'accommodates', 'bed_type', 
                'guests_included', 'cancellation_policy']
col_transformer = ColumnTransformer(transformers=[
    ('one-hot-encoder', OneHotEncoder(handle_unknown='ignore'), one_hot_cols)
], remainder='passthrough')

col_transformer.fit(train_data)

ColumnTransformer(remainder='passthrough',
                  transformers=[('one-hot-encoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['host_response_time', 'property_type',
                                  'room_type', 'accommodates', 'bed_type',
                                  'guests_included', 'cancellation_policy'])])

In [18]:
pl = Pipeline([
    ('preprocessor', col_transformer),
    ('impute_missing', SimpleImputer(strategy='constant', fill_value=-1)),
    ('linear_regression', MLPRegressor(hidden_layer_sizes=[64, 128, 128, 32], 
                                       activation='relu', learning_rate='adaptive', early_stopping=True))
])

In [23]:
train_data

,host_since,host_response_rate,host_listings_count,host_has_profile_pic,host_identity_verified,bathrooms,bedrooms,beds,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,host_response_time,property_type,room_type,accommodates,bed_type,guests_included,cancellation_policy
id,,,,,,,,,,,,,,,,,,,,
22267382,3947.0,NaN,1.0,True,True,1.0,1.0,1.0,30.0,7,12,6,100.0,NaN,Loft,Entire home/apt,2,Real Bed,2,flexible
2473861,3396.0,1.00,4.0,True,True,1.0,3.0,5.0,19.0,3,30,137,91.0,within an hour,House,Entire home/apt,8,Real Bed,3,moderate
25079703,3673.0,1.00,1.0,True,True,1.0,0.0,1.0,0.0,2,1125,3,100.0,within a few hours,Apartment,Entire home/apt,2,Real Bed,1,moderate
9342478,3316.0,NaN,1.0,True,True,1.0,1.0,1.0,0.0,3,1125,0,NaN,NaN,Apartment,Private room,1,Real Bed,1,flexible
4866426,3887.0,1.00,1.0,True,True,1.0,1.0,1.0,14.0,1,90,144,97.0,within a few hours,Apartment,Private room,2,Real Bed,1,flexible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3958032,3849.0,NaN,1.0,True,True,1.0,1.0,1.0,25.0,3,1125,19,87.0,NaN,Apartment,Private room,2,Real Bed,1,strict_14_with_grace_period
7987209,2690.0,NaN,3.0,True,True,3.0,4.0,4.0,0.0,3,365,86,95.0,NaN,Serviced apartment,Entire home/apt,8,Real Bed,1,strict_14_with_grace_period
796128,3895.0,NaN,1.0,True,True,NaN,1.0,1.0,0.0,180,365,0,NaN,NaN,House,Private room,1,Real Bed,1,flexible


In [19]:
pl.fit(train_data, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['host_response_time',
                                                   'property_type', 'room_type',
                                                   'accommodates', 'bed_type',
                                                   'guests_included',
                                                   'cancellation_policy'])])),
                ('impute_missing',
                 SimpleImputer(fill_value=-1, strategy='constant')),
                ('linear_regression',
                 MLPRegressor(early_stopping=True,
                              hidden_layer_sizes=[64, 128, 128, 32],
                              learning_rate='adaptive'))])

In [8]:
y_pred = pl.predict(train_data)
mean_squared_error(y, y_pred, squared=False)

55600.16096646746

In [9]:
raw_test_data = pd.read_csv('test.csv', parse_dates=['host_since'])
test_data = retrieve_features(raw_test_data)
test_data

C:\Users\jyson\AppData\Local\Temp\ipykernel_14672\915683724.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_test_data = pd.read_csv('test.csv', parse_dates=['host_since'])


,host_since,host_response_rate,host_listings_count,host_has_profile_pic,host_identity_verified,bathrooms,bedrooms,beds,extra_people,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,host_response_time,property_type,room_type,accommodates,bed_type,guests_included,cancellation_policy
id,,,,,,,,,,,,,,,,,,,,
19307997,2223.0,NaN,1.0,True,True,2.0,2.0,2.0,0.0,2,1125,0,NaN,NaN,Condominium,Entire home/apt,4,Real Bed,1,flexible
20176193,3326.0,0.90,1.0,True,False,1.0,2.0,1.0,0.0,2,5,22,97.0,within a day,Apartment,Entire home/apt,2,Real Bed,1,flexible
19485371,2410.0,NaN,1.0,True,True,1.0,1.0,1.0,20.0,1,3,6,100.0,NaN,Apartment,Private room,1,Real Bed,1,flexible
13079990,2554.0,NaN,1.0,True,False,1.0,1.0,1.0,0.0,1,4,7,83.0,NaN,Apartment,Private room,2,Real Bed,1,flexible
22339757,1908.0,1.00,12.0,True,False,2.0,1.0,2.0,10.0,2,1125,14,74.0,within an hour,Apartment,Private room,3,Real Bed,2,strict_14_with_grace_period
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22325617,2566.0,NaN,1.0,True,False,1.0,0.0,1.0,0.0,3,6,0,NaN,NaN,Apartment,Private room,1,Real Bed,1,flexible
8372650,2925.0,NaN,1.0,True,True,1.0,1.0,2.0,50.0,3,5,0,NaN,NaN,Apartment,Entire home/apt,3,Real Bed,2,strict_14_with_grace_period
3812554,3070.0,0.96,3.0,True,True,1.0,1.0,1.0,0.0,2,1125,67,98.0,within a few hours,House,Private room,2,Real Bed,1,moderate


In [82]:
# Save predictions to CSV file
def save_predictions(model):
    ids = test_data.index
    pred = model.predict(test_data)
    
    out = pd.DataFrame(data={'Id': ids, 'Predicted': pred})
    out.to_csv('predictions.csv', index=False)
    

In [83]:
# save_predictions(pl)